In [ ]:
# INSTALLINGS

!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0
!pip install plotly==5.3.1
!pip install scikit-learn
!pip install imagecodecs

     |████████████████████████████████| 421.8 MB 23 kB/s 
     |████████████████████████████████| 3.8 MB 33.6 MB/s 
     |████████████████████████████████| 448 kB 43.4 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=1e99f50e0e50747906a48691b8bc44fa49c84ed4634c7ace7da8dad17c7aa3bc
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
!nvidia-smi

Tue Feb 22 12:08:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi

Mon Feb 21 08:35:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# LINKAGE TO GOOGLE DRIVE AND LIBRERIES IMPORTING

from google.colab import drive
drive.mount('/content/drive')

import os
import random
import numpy as np
import plotly.express as px
import imagecodecs

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.segmentation import mark_boundaries
from scipy import ndimage
from skimage.util import img_as_float,img_as_ubyte, crop
from skimage.morphology import binary_dilation
from sklearn.utils import shuffle

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from segmentation_models import Unet

Mounted at /content/drive


Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [ ]:
# DATASET UNRAR: LOADING DATASET IN COLAB

!pip install unrar
!unrar x "drive/MyDrive/cytology challenge condivisa/00_DATASET/train.rar"     # unraring training set
!unrar x "drive/MyDrive/cytology challenge condivisa/00_DATASET/validation.rar"   # unraring validation set
!unrar x "drive/MyDrive/cytology challenge condivisa/00_DATASET/test.rar"         # unraring test set


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/cytology challenge condivisa/00_DATASET/train.rar

Creating    train                                                     OK
Creating    train/images                                              OK
Extracting  train/images/102.tif                                           0%  OK 
Extracting  train/images/104.tif                                           0%  1%  OK 
Extracting  train/images/106.tif                                           1%  OK 
Extracting  train/images/108.tif                                           1%  2%  OK 
Extracting  train/images/109.tif                                           2%  OK 
Extracting  train/images/111.tif                                           2%  3%  OK 
Extracting  train/images/112.tif                                           3%  OK 
Extracting  train/images/114.tif    

In [ ]:
# LOAD PRE-PROCESSED TRAINING SET AND VALIDATION SET (READY TO USE) and SET UP NET'S SETTINGS

# load
images = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/NORM_IL.npz')  # images loading <--------------------------------------- CHANGE HERE
labels = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/IL1_manual_mask_512x512.npz')  # manual annotations loading <-------- CHANGE HERE
X_tr = images['X_tr']
X_vl = images['X_vl']
Y_tr = labels['Y_tr']
Y_vl = labels['Y_vl']
images.close()
labels.close()
del images,labels

# settings
current_net = 'DN_NORM_MAG1'  # name of the model that will be trained <-------------------------------------------------------------- CHANGE HERE
rsz = X_tr.shape[1]  # resizing size (resize images to rsz x rsz)
NUM_CLASSES = Y_tr.max() + 1 # number of classes choosen for the problem

# conversion of labels to categorical: to minimize required space for storage
# labels has been saved as greyscale images and not as categorical labels,
# conversion is done here
Y_tr = to_categorical(Y_tr, num_classes = NUM_CLASSES, dtype='float32')  # conversion to categorical data
Y_vl = to_categorical(Y_vl, num_classes = NUM_CLASSES, dtype='float32')  # conversion to categorical data



In [ ]:
# #controllo nomi
# !unrar x "drive/MyDrive/cytology challenge condivisa/03_PREDICTED/DN_NORM_IL1.rar"
# ila = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/NORM_IL.npz')  # images loading <--------------------------------------- CHANGE HERE
# masc_bu = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/PRA_manual_mask_512x512.npz')  # manual annotations loading <-------- CHANGE HERE
# bu = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/PRA1.npz')  # images loading <--------------------------------------- CHANGE HERE

# GESU = sorted(os.listdir('DN_NORM_IL1/train/prob_map'))
# print(GESU)

In [ ]:
# CREATING DIRECTORIES IN WHICH TO STORE THE PREDICTED PROBABILITY MAPS

# prediction directories
if not os.path.exists(current_net):
  os.mkdir(current_net)

if not os.path.exists(current_net + "/train"):
  os.mkdir(current_net + "/train")

if not os.path.exists(current_net + "/validation"):
  os.mkdir(current_net + "/validation")

if not os.path.exists(current_net + "/test"):
  os.mkdir(current_net + "/test")



# probability maps directories
path_predicted_tr_pm = os.path.join(current_net,"train","prob_map")
if not os.path.exists(path_predicted_tr_pm):
  os.mkdir(path_predicted_tr_pm)

path_predicted_vl_pm = os.path.join(current_net,"validation","prob_map")
if not os.path.exists(path_predicted_vl_pm):
  os.mkdir(path_predicted_vl_pm)

path_predicted_ts_pm = os.path.join(current_net,"test","prob_map")
if not os.path.exists(path_predicted_ts_pm):
  os.mkdir(path_predicted_ts_pm)


In [ ]:
# # DATA AUGMENTATION SETTINGS

# # def coloraug(image):

# #       if image.dtype == 'float32':
# #         image = image
# #       elif image.dtype == 'uint8':
# #         image = np.array(image)
# #         #image = tf.image.random_hue(image,0.2)
# #         #image = tf.image.random_saturation(image,0.5,1.5)
# #         image = tf.image.random_contrast(image,0.8,1.2)
# #         image = image.numpy()
# #       return image


# # training set
# image_datagen = ImageDataGenerator(
#                                   # rotation_range = 180,
#                                   horizontal_flip = True,
#                                   vertical_flip = True,
#                                   fill_mode = 'constant',
#                                   cval = 0,
#                                   zoom_range = [0.7,1.3],
#                                   brightness_range=[0.8,1.2],
#                                   #preprocessing_function = coloraug
# )


# # validation set
# val_datagen = ImageDataGenerator()

# # Generator function
# def XYaugmentGenerator(X1, y, batch_sz, sd):
#         genX1 = image_datagen.flow(X1, y, batch_size=batch_sz,seed = sd)
#         genX2 = image_datagen.flow(y, X1, batch_size=batch_sz, seed = sd)
#         while True:
#             X1i = genX1.next()  #The next() function returns the next item in an iterator.
#             X2i = genX2.next()
#             yield X1i[0], X2i[0]

In [ ]:
# # NET TRAINING

# # defining U-NET architecture
# BACKBONE = 'densenet201'
# model = Unet(backbone_name = BACKBONE,
#             input_shape = (rsz,rsz,3),
#             encoder_weights = 'imagenet',
#             encoder_freeze = True,
#             decoder_block_type = 'transpose',
#             classes = NUM_CLASSES,
#             activation = 'softmax')

# # defining optimization algorithm and loss function
# model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# # training and parameters <---------------------------------------------------------------------- CHANGE HERE (below)
# n_epochs = 20
# batch_sz = 2
# sd=1
# monitor_value = 'val_categorical_accuracy'
# verbose_value = 1
# min_delta_value = 0.001
# patience_value = 10
# mode_value = 'max'

# # checkpoint definition
# csv_logger = CSVLogger('./log.out', append=True, separator=';')
# earlystopping = EarlyStopping(monitor = monitor_value, verbose = verbose_value, min_delta = min_delta_value, patience = patience_value, mode = mode_value, restore_best_weights = True)
# callbacks_list = [csv_logger, earlystopping]


# # model training
# results = model.fit_generator(XYaugmentGenerator(X_tr,Y_tr, batch_sz, sd),
#                               steps_per_epoch = np.ceil(float(len(X_tr))/float(batch_sz)),
#                               validation_data = val_datagen.flow(X_vl,Y_vl,batch_sz),
#                               validation_steps = np.ceil(float(len(X_vl))/float(batch_sz)),
#                               shuffle = True,
#                               epochs = n_epochs,
#                               callbacks = callbacks_list)

74842112/74836368 [==============================] - 3s 0us/step
Epoch 1/20
150/150 [==============================] - 255s 2s/step - loss: 93.5068 - categorical_accuracy: 0.8120 - val_loss: 0.2296 - val_categorical_accuracy: 0.8952
Epoch 2/20
150/150 [==============================] - 157s 1s/step - loss: 45.7195 - categorical_accuracy: 0.9140 - val_loss: 0.2170 - val_categorical_accuracy: 0.8964
Epoch 3/20
150/150 [==============================] - 157s 1s/step - loss: 40.5594 - categorical_accuracy: 0.8955 - val_loss: 0.1126 - val_categorical_accuracy: 0.9367
Epoch 4/20
150/150 [==============================] - 156s 1s/step - loss: 36.8284 - categorical_accuracy: 0.9144 - val_loss: 0.1571 - val_categorical_accuracy: 0.9489
Epoch 5/20
150/150 [==============================] - 156s 1s/step - loss: 35.5354 - categorical_accuracy: 0.9273 - val_loss: 0.1928 - val_categorical_accuracy: 0.9453
Epoch 6/20
150/150 [==============================] - 156s 1s/step - loss: 33.2639 - categorica

In [ ]:
# SAVE/LOAD OF TRAINED MODEL

if not os.path.exists('drive/MyDrive/cytology challenge condivisa/02_TRAINED MODELS/'):
    os.mkdir('drive/MyDrive/cytology challenge condivisa/02_TRAINED MODELS/')

#model.save('drive/MyDrive/cytology challenge condivisa/02_TRAINED MODELS/' + current_net)
model = load_model('drive/MyDrive/cytology challenge condivisa/02_TRAINED MODELS/' + current_net)

In [ ]:
# PROBABILITY MAP PREDICTIONS ON TRAINING SET

# path
tr_IMGS_path = os.path.join('train','images')

# cycling on all images to apply the U-NET
tr_images = sorted(os.listdir(tr_IMGS_path))

for n,id_ in tqdm(enumerate(tr_images), total=len(tr_images)):

    img = np.reshape(img_as_ubyte(X_tr[n]),(1,rsz,rsz,3))  # load and resizing (and resahping to feed for net format) of stained image
    softmax = model.predict(img)  # apply CNN for prediction

    imsave(os.path.join(path_predicted_tr_pm,id_),softmax)  # predicted heatmap saving

100%|██████████| 300/300 [01:13<00:00,  4.10it/s]


In [ ]:
# PROBABILITY MAP PREDICTIONS ON VALIDATION SET

# Path
vl_IMGS_path = os.path.join('validation','images')

# cycling on all images to apply the U-NET
vl_images = sorted(os.listdir(vl_IMGS_path))

for n, id_ in tqdm(enumerate(vl_images), total=len(vl_images)):

    img = np.reshape(img_as_ubyte(X_vl[n]),(1,rsz,rsz,3))  # load and resizing (and resahping to feed for net format) of stained image
    softmax = model.predict(img)  # apply CNN for prediction

    imsave(os.path.join(path_predicted_vl_pm,id_),softmax)  # predicted heatmap saving

100%|██████████| 50/50 [00:09<00:00,  5.30it/s]


In [ ]:
# SUL TEST DEVO APPLICARE LO STESSO PRE-PROCESSING PRIMA
# # PROBABILITY MAP PREDICTIONS ON TEST SET

# # Path
# ts_IMGS_path = os.path.join('test','images')

# # cycling on all images to apply the U-NET
# ts_images = os.listdir(ts_IMGS_path)
# for n, id_ in tqdm(enumerate(ts_images), total=len(ts_images)):

#     img = np.reshape(img_as_ubyte(resize(imread(ts_IMGS_path+'/'+id_),(rsz,rsz))),(1,rsz,rsz,3))  # load and resizing (and resahping to feed for net format) of stained image
#     softmax = model.predict(img)  # apply CNN for prediction

#     imsave(os.path.join(path_predicted_ts_pm,id_),softmax)  # predicted heatmap saving

In [ ]:
#SALVATAGGIO RISULTATI COMPRESSI COME .rar DA COLAB A DRIVE

#compressione
!apt-get install rar  # installing necessary library
!rar a "DN_NORM_MAG1" "DN_NORM_MAG1" # <------------------------------------------------------------------------- CHANGE HERE

#copia da colab a drive (cartella "03_PREDICTED" in "cytology challenge")

import shutil
shutil.copyfile(current_net + ".rar", "/content/drive/MyDrive/cytology challenge condivisa/03_PREDICTED/" + current_net + ".rar")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

'/content/drive/MyDrive/cytology challenge condivisa/03_PREDICTED/DN_NORM_MAG1.rar'